In [84]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, SimpleRNN
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../utils')
import KMeansDateRange
import get_open_close
import timeseries

"""K is the number of different
industries/sectors in the stock market"""
k = 11

In [85]:
df = get_open_close.populate_values_efficient("2012-05-18", "2020-04-01")

In [86]:
clusters = KMeansDateRange.kmeans_automated("2012-05-18", "2020-04-01", k=k)

In [88]:
%%time
mean_absolute_loss = 0

"""Randomly sampling 20 stocks from each cluster"""
for cluster in range(0, k):
    
    cluster_mae = 0
    sampled_stocks = clusters.loc[clusters['clusters'] == cluster].sample(n=20, random_state=1337)
    
    model = Sequential()
    model.add(SimpleRNN(128, activation='relu', input_shape=(14, 2), return_sequences=True))
    model.add(SimpleRNN(128, activation='relu', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation="linear"))
    model.compile(optimizer='adam', loss='mae')
    
    stock_x_test = {}
    
    """Fit the data of each stock within a cluster on the RNN"""
    for index, stock in enumerate(sampled_stocks['symbols']):
        stock_data = df[stock]
        
        x_train, x_test = train_test_split(stock_data, test_size=0.3, shuffle=False)
        
        scaler = StandardScaler()
        x_train= scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)
        stock_x_test[stock] = x_test

        x_train, y_train = timeseries.get_timeseries_X_y(x_train)
        
        model.fit(x_train, y_train, validation_split=0.3, verbose=1,
                            initial_epoch=index*20, epochs=(index + 1)*20)
        
    for stock in stock_x_test:
        x_test, y_test = timeseries.get_timeseries_X_y(stock_x_test[stock])
        loss = model.evaluate(x_test, y_test)
        cluster_mae = cluster_mae + loss
    cluster_mae = cluster_mae/(sampled_stocks.shape[0])
    mean_absolute_loss = mean_absolute_loss + cluster_mae

print("Loss of Vanilla RNN: ", mean_absolute_loss/k)

Epoch 1/20
30/30 [==============================] - 2s 22ms/step - loss: 0.1854 - val_loss: 0.0345
Epoch 2/20
30/30 [==============================] - 0s 11ms/step - loss: 0.1200 - val_loss: 0.0175
Epoch 3/20
30/30 [==============================] - 0s 11ms/step - loss: 0.1029 - val_loss: 0.0681
Epoch 4/20
30/30 [==============================] - 0s 11ms/step - loss: 0.1058 - val_loss: 0.0606
Epoch 5/20
30/30 [==============================] - 0s 10ms/step - loss: 0.1001 - val_loss: 0.0183
Epoch 6/20
30/30 [==============================] - 0s 10ms/step - loss: 0.0893 - val_loss: 0.0316
Epoch 7/20
30/30 [==============================] - 0s 10ms/step - loss: 0.0891 - val_loss: 0.0747
Epoch 8/20
30/30 [==============================] - 0s 10ms/step - loss: 0.0857 - val_loss: 0.0752
Epoch 9/20
30/30 [==============================] - 0s 10ms/step - loss: 0.0948 - val_loss: 0.0346
Epoch 10/20
30/30 [==============================] - 0s 11ms/step - loss: 0.0847 - val_loss: 0.0515
Epoch 11/

30/30 [==============================] - 0s 10ms/step - loss: 0.0944 - val_loss: 0.0278
Epoch 84/100
30/30 [==============================] - 0s 9ms/step - loss: 0.0870 - val_loss: 0.0471
Epoch 85/100
30/30 [==============================] - 0s 9ms/step - loss: 0.0872 - val_loss: 0.0279
Epoch 86/100
30/30 [==============================] - 0s 10ms/step - loss: 0.0908 - val_loss: 0.0270
Epoch 87/100
30/30 [==============================] - 0s 9ms/step - loss: 0.0866 - val_loss: 0.0238
Epoch 88/100
30/30 [==============================] - 0s 10ms/step - loss: 0.0857 - val_loss: 0.0311
Epoch 89/100
30/30 [==============================] - 0s 10ms/step - loss: 0.0949 - val_loss: 0.0284
Epoch 90/100
30/30 [==============================] - 0s 10ms/step - loss: 0.0898 - val_loss: 0.0236
Epoch 91/100
30/30 [==============================] - 0s 9ms/step - loss: 0.0905 - val_loss: 0.0297
Epoch 92/100
30/30 [==============================] - 0s 10ms/step - loss: 0.0842 - val_loss: 0.0237
Epoch 9

30/30 [==============================] - 0s 8ms/step - loss: 0.0859 - val_loss: 0.0423
Epoch 165/180
30/30 [==============================] - 0s 9ms/step - loss: 0.0859 - val_loss: 0.0615
Epoch 166/180
30/30 [==============================] - 0s 9ms/step - loss: 0.0864 - val_loss: 0.0498
Epoch 167/180
30/30 [==============================] - 0s 8ms/step - loss: 0.0814 - val_loss: 0.0442
Epoch 168/180
30/30 [==============================] - 0s 8ms/step - loss: 0.0854 - val_loss: 0.0393
Epoch 169/180
30/30 [==============================] - 0s 8ms/step - loss: 0.0821 - val_loss: 0.0433
Epoch 170/180
30/30 [==============================] - 0s 8ms/step - loss: 0.0835 - val_loss: 0.0462
Epoch 171/180
30/30 [==============================] - 0s 8ms/step - loss: 0.0860 - val_loss: 0.0467
Epoch 172/180
30/30 [==============================] - 0s 8ms/step - loss: 0.0825 - val_loss: 0.0416
Epoch 173/180
30/30 [==============================] - 0s 8ms/step - loss: 0.0817 - val_loss: 0.0457
Epoc

30/30 [==============================] - 0s 9ms/step - loss: 0.1942 - val_loss: 0.2119
Epoch 246/260
30/30 [==============================] - 0s 8ms/step - loss: 0.1869 - val_loss: 0.2073
Epoch 247/260
30/30 [==============================] - 0s 8ms/step - loss: 0.1914 - val_loss: 0.2094
Epoch 248/260
30/30 [==============================] - 0s 8ms/step - loss: 0.1847 - val_loss: 0.2110
Epoch 249/260
30/30 [==============================] - 0s 9ms/step - loss: 0.1880 - val_loss: 0.1966
Epoch 250/260
30/30 [==============================] - 0s 9ms/step - loss: 0.1871 - val_loss: 0.2088
Epoch 251/260
30/30 [==============================] - 0s 8ms/step - loss: 0.1799 - val_loss: 0.2169
Epoch 252/260
30/30 [==============================] - 0s 8ms/step - loss: 0.1734 - val_loss: 0.2548
Epoch 253/260
30/30 [==============================] - 0s 9ms/step - loss: 0.1710 - val_loss: 0.2220
Epoch 254/260
30/30 [==============================] - 0s 9ms/step - loss: 0.1656 - val_loss: 0.2369
Epoc

30/30 [==============================] - 0s 9ms/step - loss: 0.0835 - val_loss: 0.0149
Epoch 327/340
30/30 [==============================] - 0s 9ms/step - loss: 0.0803 - val_loss: 0.0659
Epoch 328/340
30/30 [==============================] - 0s 8ms/step - loss: 0.0846 - val_loss: 0.0588
Epoch 329/340
30/30 [==============================] - 0s 9ms/step - loss: 0.0828 - val_loss: 0.0158
Epoch 330/340
30/30 [==============================] - 0s 9ms/step - loss: 0.0823 - val_loss: 0.0294
Epoch 331/340
30/30 [==============================] - 0s 9ms/step - loss: 0.0818 - val_loss: 0.0536
Epoch 332/340
30/30 [==============================] - 0s 8ms/step - loss: 0.0875 - val_loss: 0.0150
Epoch 333/340
30/30 [==============================] - 0s 9ms/step - loss: 0.0840 - val_loss: 0.0244
Epoch 334/340
30/30 [==============================] - 0s 9ms/step - loss: 0.0852 - val_loss: 0.0362
Epoch 335/340
30/30 [==============================] - 0s 9ms/step - loss: 0.0822 - val_loss: 0.0312
Epoc

19/19 [==============================] - 0s 3ms/step - loss: 0.1033
Loss of Vanilla RNN:  0.10406925340066664
CPU times: user 4min 10s, sys: 44.8 s, total: 4min 54s
Wall time: 1min 53s
